# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,calama,-22.4667,-68.9333,21.27,5,0,9.26,CL,1728687613
1,1,aswan,24.0934,32.9070,27.61,21,0,2.06,EG,1728687635
2,2,west island,-12.1568,96.8225,25.99,89,40,6.17,CC,1728687726
3,3,quepos,9.4236,-84.1652,26.15,89,100,0.86,CR,1728687728
4,4,invercargill,-46.4000,168.3500,13.90,61,55,4.02,NZ,1728687729


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [14]:
import warnings
warnings.filterwarnings("ignore")

In [32]:
%%capture --no-display

# Configure the map plot
citiesmap = city_data_df.hvplot.points('Lng','Lat',geo = True,tiles = 'OSM',frame_width = 700,
    frame_height = 500,size = 'Humidity',color = 'City')

# Display the map
citiesmap

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [35]:
#A max temperature lower than 27 degrees but higher than 21
#Wind speed less than 4.5 m/s
#Zero cloudiness
# Narrow down cities that fit criteria and drop any results with null values
vaca_df = city_data_df.loc[(city_data_df["Max Temp"] >=21) & (city_data_df['Max Temp']<= 27)& (city_data_df['Wind Speed'] <4.5)&(city_data_df['Cloudiness'] == 0)]

# Drop any rows with null values
vaca_df = vaca_df.dropna(how = 'any')

# Display sample data
vaca_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
27,27,muroto-misakicho,33.2833,134.1500,21.42,61,0,4.32,JP,1728687759
61,61,mersa matruh,31.3525,27.2453,23.11,83,0,0.00,EG,1728687802
117,117,kharan,28.5833,65.4167,24.52,12,0,3.48,PK,1728687873
194,194,padampur,29.6833,73.6167,25.55,25,0,2.06,IN,1728687975
267,267,kingaroy,-26.5333,151.8333,21.05,55,0,1.92,AU,1728688124
398,398,tazacorte,28.6290,-17.9293,22.54,75,0,2.57,ES,1728688297
407,407,conde,-7.2597,-34.9075,25.70,69,0,3.60,BR,1728688308
410,410,ishikawa,26.4233,127.8214,25.19,78,0,1.54,JP,1728688312
429,429,al kharijah,25.4514,30.5464,24.72,32,0,2.58,EG,1728688336
480,480,orapa,-21.2833,25.3667,21.62,28,0,3.90,BW,1728688407


### Step 3: Create a new DataFrame called `hotel_df`.

In [37]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = vaca_df[['City','Country','Lat','Lng','Humidity']]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
27,muroto-misakicho,JP,33.2833,134.1500,61,
61,mersa matruh,EG,31.3525,27.2453,83,
117,kharan,PK,28.5833,65.4167,12,
194,padampur,IN,29.6833,73.6167,25,
267,kingaroy,AU,-26.5333,151.8333,55,
398,tazacorte,ES,28.6290,-17.9293,75,
407,conde,BR,-7.2597,-34.9075,69,
410,ishikawa,JP,26.4233,127.8214,78,
429,al kharijah,EG,25.4514,30.5464,32,
480,orapa,BW,-21.2833,25.3667,28,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [40]:
# Set parameters to search for a hotel
radius = 10000
params = { 
"categories":"accommodation.hotel",
"limit":20,
"apiKey":geoapify_key 
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lati = row['Lat']
    longi = row['Lng']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longi},{lati},{radius}"
    params["bias"] = f"proximity:{longi},{lati}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url,params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
muroto-misakicho - nearest hotel: 民宿うらしま
mersa matruh - nearest hotel: صقور
kharan - nearest hotel: No hotel found
padampur - nearest hotel: No hotel found
kingaroy - nearest hotel: No hotel found
tazacorte - nearest hotel: App Leyma
conde - nearest hotel: Paladino
ishikawa - nearest hotel: サンセットヒル
al kharijah - nearest hotel: Kharga Hotel
orapa - nearest hotel: No hotel found
khangarh - nearest hotel: No hotel found
al wajh - nearest hotel: فندق رالين 2
demopolis - nearest hotel: No hotel found
tabas - nearest hotel: هتل امیر
rudbar - nearest hotel: No hotel found
eilat - nearest hotel: Melony Club
holdrege - nearest hotel: Hotel Dale


,City,Country,Lat,Lng,Humidity,Hotel Name
27,muroto-misakicho,JP,33.2833,134.1500,61,民宿うらしま
61,mersa matruh,EG,31.3525,27.2453,83,صقور
117,kharan,PK,28.5833,65.4167,12,No hotel found
194,padampur,IN,29.6833,73.6167,25,No hotel found
267,kingaroy,AU,-26.5333,151.8333,55,No hotel found
398,tazacorte,ES,28.6290,-17.9293,75,App Leyma
407,conde,BR,-7.2597,-34.9075,69,Paladino
410,ishikawa,JP,26.4233,127.8214,78,サンセットヒル
429,al kharijah,EG,25.4514,30.5464,32,Kharga Hotel
480,orapa,BW,-21.2833,25.3667,28,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [46]:
%%capture --no-display

# Configure the map plot
hotelmap = hotel_df.hvplot.points('Lng','Lat',geo = True,tiles = 'OSM',size = 'Humidity',color = 'City',hover_cols=['Hotel Name','Country'])

# Display the map
hotelmap

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)